# Default Import

In [1]:
from ris import pysqldb
from IPython.display import Markdown, clear_output
from sqlalchemy import create_engine
import ris
import getpass
import datetime 
import pandas as pd
import numpy as np
import os
import requests

timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

Markdown('<strong>Notebook run on: {} | by {} | Using ris library version: {} | File Location: {}'.format(
    timestamp, getpass.getuser(), ris.__version__, os.getcwd()
))


<strong>Notebook run on: 2020-09-02 17:38 | by soge | Using ris library version: 1.5.6 | File Location: E:\RIS\Staff Folders\Samuel\Requests\AG\SIP_treatment_request_83120

# Db connections

In [2]:
try: 
    db.params['user']
except:
    db = pysqldb.DbConnect(type='PG', server='dotpgsql01', database='sip')


Additional database connection details required:
User name (sip):soge
Password (sip)········
Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-09-02 17:39:01


In [3]:
#sip = pysqldb.DbConnect(server='dotpgsql01', database='sip', type='PG', user = db.params['user'], password = db.params['password'])
#forms = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

In [4]:
db.connect()

Database connection (PG) to sip on dotpgsql01 - user: soge 
Connection established 2020-09-02 17:39:02


In [5]:
sip_treatments = db.dfquery(""" -- Grab pid, tid, treatment name, flattened tree treatment type, and project install dates,  
                                --      treatment install dates, project end dates, treatment end dates,  mileage of treatment, 
                                --      pm, and sip year
                                -- sip_treatments join to sip_projects on pid_fk
                                -- Join to sip geo to get the mileage
                                -- Limited to Bikes
                                

                                WITH data AS(
                                SELECT tree.descendant AS qid, 
                                array_to_string(array_agg(tree.concat), ' -> '::text) AS full_treatment
                                FROM (  SELECT p.descendant, p.ancestor, q.question, p.depth, 
                                       pg_catalog.concat(p.ancestor, ': ', q.question, ' (Depth:', p.depth, ')') AS concat
                                    FROM sip_questions_path p
                                    JOIN sip_questions q ON p.ancestor = q.qid
                                    ORDER BY p.descendant, p.depth DESC) tree
                                GROUP BY tree.descendant
                                )


                                SELECT sp.pid, st.tid, st.treatment, data.full_treatment, sp.sip_year, sp.start_date project_start, 
                                       st.start_date treatment_start, sp.end_date project_end, st.end_date treatment_end, sl.description,
                                       sum(coalesce(st_length(cl.geom)/5280,0)) mileage, sp.pm
                                FROM public.sip_treatments st
                                LEFT JOIN data
                                ON data.qid = st.breadcrumbid
                                LEFT JOIN public.sip_projects sp
                                ON st.pid_fk = sp.pid
                                LEFT JOIN public.sip_treatments_geo stg
                                ON st.tid = stg.tid_fk
                                LEFT JOIN public.clion cl
                                ON stg.segmentid::int = cl.segmentid::int
                                JOIN sip_lookup sl
                                ON sp.status::int = sl.lookupid
                                WHERE (((sp.status::int = 15 or sp.status::int = 11) and sip_year = 2020) 
                                OR (sp.status::int = 11 and sip_year = 2021))
                                AND sp.unit = '24'
                                GROUP BY sp.pid, st.tid, st.treatment, data.full_treatment, sp.sip_year, sp.start_date, 
                                     st.start_date, sp.end_date, st.end_date, sl.description, sp.pm


                                """)

In [6]:
#sip_treatments

In [7]:
sip_treatments.to_csv('sip_treatments_{ts}.csv'.format(ts=ts), index = False)